# AffectNet prep

In [1]:
import json

In [36]:
with open('splits/new/merged_data_splits.json', 'r') as f:
    casia_oulucasia_split = json.loads(f.read())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
# Load the existing data from the two files
with open('splits/new/casia2_data_splits.json', 'r') as f:
    casia2_data = json.load(f)
with open('splits/new/oulucasia_data_splits.json', 'r') as f:
    oulucasia_data = json.load(f)

In [7]:
# Merge the data
merged_data = {}
for key in set(casia2_data.keys()).union(oulucasia_data.keys()):
    if key != 'metadata':
        merged_data[key] = casia2_data.get(key, []) + oulucasia_data.get(key, [])

# Merge the metadata
merged_metadata = {}
for key in set(casia2_data['metadata'].keys()).union(oulucasia_data['metadata'].keys()):
    merged_metadata[key] = casia2_data['metadata'].get(key, 0) + oulucasia_data['metadata'].get(key, 0)

# Add the merged metadata to the merged data
merged_data['metadata'] = merged_metadata

In [37]:
merged_data['nir_rest']

['data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00090_003.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10168_014.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00158_029.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_00118_009.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s3_NIR_20464_014.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00164_025.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s4_NIR_30780_013.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s3_NIR_20478_013.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s3_NIR_20393_011.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10087_013.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10177_011.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10069_007.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10245_017.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00190_015.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10129_003.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s2_NIR_10198_17-B.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00136_014.bmp',
 'data/CASIA_NISVIR/NIR-VIS/NIR/s1_NIR_00170_003.bmp',
 'data/CA

In [49]:
import json
import pathlib
import os
import click
import cv2

from deepface import DeepFace


def detect_and_align_face(image_fp):
    try:
        face_objs = DeepFace.extract_faces(
            img_path=image_fp,
            target_size=(256, 256),
            detector_backend="retinaface",
            enforce_detection=False,
        )
    except Exception as e:
        print(f"ERROR at {image_fp}", e)
        return None

    if len(face_objs) != 1:
        print("NOT FOUND OR MULTIPLE FACES!")
        return None

    face = face_objs[0]["face"]

    return face

def preprocess_part( fps, target_fp, spectra):
    # prepare filepath
    os.makedirs(target_fp, exist_ok=True)

    # align faces for all images
    i = 0
    preprocessed_fps = []
    for fp in fps:
        new_filename = "-".join(pathlib.PurePath(fp).parts[-3:])
        target_path = os.path.join(target_fp, new_filename)

        aligned_face = detect_and_align_face(fp)

        if aligned_face is None:
            continue

        aligned_face = 255 * aligned_face[:, :, ::-1]

        cv2.imwrite(target_path, aligned_face)

        print(f"#{i} {spectra} Stored: {new_filename}")
        i += 1
        preprocessed_fps.append(target_path)

    return preprocessed_fps

In [51]:
preproc_casia_fps = preprocess_part(merged_data['nir_rest'], "data/for_unpaired-casia_preprocessed_rest_of_NIR/", "NIR")

2023-12-14 23:49:05.049512: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89522176 exceeds 10% of free system memory.
2023-12-14 23:49:05.355692: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90135040 exceeds 10% of free system memory.
2023-12-14 23:49:05.429889: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89653248 exceeds 10% of free system memory.
2023-12-14 23:49:05.498660: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89653248 exceeds 10% of free system memory.
2023-12-14 23:49:05.546168: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89653248 exceeds 10% of free system memory.


#0 NIR Stored: NIR-VIS-NIR-s1_NIR_00090_003.bmp
#1 NIR Stored: NIR-VIS-NIR-s2_NIR_10168_014.bmp
#2 NIR Stored: NIR-VIS-NIR-s1_NIR_00158_029.bmp
#3 NIR Stored: NIR-VIS-NIR-s2_NIR_00118_009.bmp
#4 NIR Stored: NIR-VIS-NIR-s3_NIR_20464_014.bmp
#5 NIR Stored: NIR-VIS-NIR-s1_NIR_00164_025.bmp
#6 NIR Stored: NIR-VIS-NIR-s4_NIR_30780_013.bmp
#7 NIR Stored: NIR-VIS-NIR-s3_NIR_20478_013.bmp
#8 NIR Stored: NIR-VIS-NIR-s3_NIR_20393_011.bmp
#9 NIR Stored: NIR-VIS-NIR-s2_NIR_10087_013.bmp
#10 NIR Stored: NIR-VIS-NIR-s2_NIR_10177_011.bmp
#11 NIR Stored: NIR-VIS-NIR-s2_NIR_10069_007.bmp
#12 NIR Stored: NIR-VIS-NIR-s2_NIR_10245_017.bmp
#13 NIR Stored: NIR-VIS-NIR-s1_NIR_00190_015.bmp
#14 NIR Stored: NIR-VIS-NIR-s2_NIR_10129_003.bmp
#15 NIR Stored: NIR-VIS-NIR-s2_NIR_10198_17-B.bmp
#16 NIR Stored: NIR-VIS-NIR-s1_NIR_00136_014.bmp
#17 NIR Stored: NIR-VIS-NIR-s1_NIR_00170_003.bmp
#18 NIR Stored: NIR-VIS-NIR-s3_NIR_20372_008.bmp
#19 NIR Stored: NIR-VIS-NIR-s1_NIR_00059_011.bmp
#20 NIR Stored: NIR-VIS-NIR-s

#166 NIR Stored: NIR-VIS-NIR-s3_NIR_20379_017.bmp
#167 NIR Stored: NIR-VIS-NIR-s2_NIR_10029_001.bmp
#168 NIR Stored: NIR-VIS-NIR-s2_NIR_10015_005.bmp
#169 NIR Stored: NIR-VIS-NIR-s2_NIR_10124_007.bmp
#170 NIR Stored: NIR-VIS-NIR-s2_NIR_10236_015.bmp
#171 NIR Stored: NIR-VIS-NIR-s4_NIR_30776_013.bmp
#172 NIR Stored: NIR-VIS-NIR-s1_NIR_00072_004.bmp
#173 NIR Stored: NIR-VIS-NIR-s1_NIR_00084_001.bmp
#174 NIR Stored: NIR-VIS-NIR-s2_NIR_10091_017.bmp
#175 NIR Stored: NIR-VIS-NIR-s2_NIR_10097_009.bmp
#176 NIR Stored: NIR-VIS-NIR-s1_NIR_00048_005.bmp
#177 NIR Stored: NIR-VIS-NIR-s1_NIR_00095_003.bmp
#178 NIR Stored: NIR-VIS-NIR-s3_NIR_20336_002.bmp
#179 NIR Stored: NIR-VIS-NIR-s3_NIR_20496_013.bmp
#180 NIR Stored: NIR-VIS-NIR-s3_NIR_20421_010.bmp
#181 NIR Stored: NIR-VIS-NIR-s2_NIR_10017_017.bmp
#182 NIR Stored: NIR-VIS-NIR-s2_NIR_10202_08-B.bmp
#183 NIR Stored: NIR-VIS-NIR-s2_NIR_10069_003.bmp
#184 NIR Stored: NIR-VIS-NIR-s2_NIR_10272_007.bmp
#185 NIR Stored: NIR-VIS-NIR-s3_NIR_20381_008.bmp

#330 NIR Stored: NIR-VIS-NIR-s2_NIR_10285_004.bmp
#331 NIR Stored: NIR-VIS-NIR-s4_NIR_30758_013.bmp
#332 NIR Stored: NIR-VIS-NIR-s2_NIR_10062_007.bmp
#333 NIR Stored: NIR-VIS-NIR-s2_NIR_10166_001.bmp
#334 NIR Stored: NIR-VIS-NIR-s3_NIR_20477_009.bmp
#335 NIR Stored: NIR-VIS-NIR-s2_NIR_10019_013.bmp
#336 NIR Stored: NIR-VIS-NIR-s2_NIR_10197_20-B.bmp
#337 NIR Stored: NIR-VIS-NIR-s3_NIR_20320_005.bmp
#338 NIR Stored: NIR-VIS-NIR-s4_NIR_30768_016.bmp
#339 NIR Stored: NIR-VIS-NIR-s2_NIR_10256_003.bmp
#340 NIR Stored: NIR-VIS-NIR-s1_NIR_00178_009.bmp
#341 NIR Stored: NIR-VIS-NIR-s2_NIR_10077_008.bmp
#342 NIR Stored: NIR-VIS-NIR-s1_NIR_00008_002.bmp
#343 NIR Stored: NIR-VIS-NIR-s2_NIR_10270_020.bmp
#344 NIR Stored: NIR-VIS-NIR-s1_NIR_00009_004.bmp
#345 NIR Stored: NIR-VIS-NIR-s3_NIR_20448_006.bmp
#346 NIR Stored: NIR-VIS-NIR-s2_NIR_10146_009.bmp
#347 NIR Stored: NIR-VIS-NIR-s2_NIR_10227_014.bmp
#348 NIR Stored: NIR-VIS-NIR-s1_NIR_00076_002.bmp
#349 NIR Stored: NIR-VIS-NIR-s2_NIR_10202_20-B.bm

#494 NIR Stored: NIR-VIS-NIR-s3_NIR_20443_011.bmp
#495 NIR Stored: NIR-VIS-NIR-s1_NIR_00122_006.bmp
#496 NIR Stored: NIR-VIS-NIR-s2_NIR_10258_007.bmp
#497 NIR Stored: NIR-VIS-NIR-s3_NIR_20425_008.bmp
#498 NIR Stored: NIR-VIS-NIR-s3_NIR_20394_002.bmp
#499 NIR Stored: NIR-VIS-NIR-s3_NIR_20492_007.bmp
#500 NIR Stored: NIR-VIS-NIR-s3_NIR_20329_013.bmp
#501 NIR Stored: NIR-VIS-NIR-s2_NIR_10236_009.bmp
#502 NIR Stored: NIR-VIS-NIR-s2_NIR_10163_006.bmp
#503 NIR Stored: NIR-VIS-NIR-s2_NIR_10298_009.bmp
#504 NIR Stored: NIR-VIS-NIR-s2_NIR_10015_007.bmp
#505 NIR Stored: NIR-VIS-NIR-s1_NIR_00015_007.bmp
#506 NIR Stored: NIR-VIS-NIR-s2_NIR_10076_013.bmp
#507 NIR Stored: NIR-VIS-NIR-s1_NIR_00194_006.bmp
#508 NIR Stored: NIR-VIS-NIR-s1_NIR_00138_003.bmp
#509 NIR Stored: NIR-VIS-NIR-s2_NIR_10288_012.bmp
#510 NIR Stored: NIR-VIS-NIR-s2_NIR_10250_002.bmp
#511 NIR Stored: NIR-VIS-NIR-s3_NIR_20322_013.bmp
#512 NIR Stored: NIR-VIS-NIR-s1_NIR_00112_007.bmp
#513 NIR Stored: NIR-VIS-NIR-s1_NIR_00012_002.bmp


...

#6611 NIR Stored: NIR-VIS-NIR-s3_NIR_20333_007.bmp
#6612 NIR Stored: NIR-VIS-NIR-s2_NIR_10144_002.bmp
#6613 NIR Stored: NIR-VIS-NIR-s2_NIR_10173_009.bmp
#6614 NIR Stored: NIR-VIS-NIR-s3_NIR_20343_001.bmp
#6615 NIR Stored: NIR-VIS-NIR-s4_NIR_30517_001.bmp
#6616 NIR Stored: NIR-VIS-NIR-s2_NIR_10134_016.bmp
NOT FOUND OR MULTIPLE FACES!
#6617 NIR Stored: NIR-VIS-NIR-s2_NIR_10275_003.bmp
#6618 NIR Stored: NIR-VIS-NIR-s2_NIR_10285_012.bmp
#6619 NIR Stored: NIR-VIS-NIR-s1_NIR_00102_002.bmp
#6620 NIR Stored: NIR-VIS-NIR-s1_NIR_00045_005.bmp
#6621 NIR Stored: NIR-VIS-NIR-s1_NIR_00058_010.bmp
#6622 NIR Stored: NIR-VIS-NIR-s4_NIR_30565_009.bmp
#6623 NIR Stored: NIR-VIS-NIR-s2_NIR_10014_005.bmp
#6624 NIR Stored: NIR-VIS-NIR-s4_NIR_30771_020.bmp
#6625 NIR Stored: NIR-VIS-NIR-s2_NIR_00164_016.bmp
#6626 NIR Stored: NIR-VIS-NIR-s1_NIR_00172_004.bmp
#6627 NIR Stored: NIR-VIS-NIR-s2_NIR_10293_013.bmp
#6628 NIR Stored: NIR-VIS-NIR-s1_NIR_00191_017.bmp
#6629 NIR Stored: NIR-VIS-NIR-s2_NIR_10308_017.bmp
#6

#6771 NIR Stored: NIR-VIS-NIR-s2_NIR_10076_008.bmp
#6772 NIR Stored: NIR-VIS-NIR-s1_NIR_00090_007.bmp
#6773 NIR Stored: NIR-VIS-NIR-s2_NIR_10022_014.bmp
#6774 NIR Stored: NIR-VIS-NIR-s3_NIR_20487_009.bmp
#6775 NIR Stored: NIR-VIS-NIR-s2_NIR_10224_011.bmp
#6776 NIR Stored: NIR-VIS-NIR-s1_NIR_00148_007.bmp
#6777 NIR Stored: NIR-VIS-NIR-s1_NIR_00104_004.bmp
#6778 NIR Stored: NIR-VIS-NIR-s2_NIR_10037_006.bmp
#6779 NIR Stored: NIR-VIS-NIR-s1_NIR_00157_012.bmp
#6780 NIR Stored: NIR-VIS-NIR-s1_NIR_00159_003.bmp
#6781 NIR Stored: NIR-VIS-NIR-s4_NIR_30746_014.bmp
#6782 NIR Stored: NIR-VIS-NIR-s1_NIR_00080_001.bmp
#6783 NIR Stored: NIR-VIS-NIR-s1_NIR_00147_014.bmp
#6784 NIR Stored: NIR-VIS-NIR-s2_NIR_10291_011.bmp
#6785 NIR Stored: NIR-VIS-NIR-s2_NIR_10265_005.bmp
#6786 NIR Stored: NIR-VIS-NIR-s3_NIR_20406_014.bmp
#6787 NIR Stored: NIR-VIS-NIR-s4_NIR_30602_020.bmp
#6788 NIR Stored: NIR-VIS-NIR-s2_NIR_10196_06-B.bmp
#6789 NIR Stored: NIR-VIS-NIR-s2_NIR_10173_001.bmp
#6790 NIR Stored: NIR-VIS-NIR-

#6932 NIR Stored: NIR-VIS-NIR-s3_NIR_20471_012.bmp
#6933 NIR Stored: NIR-VIS-NIR-s3_NIR_20461_005.bmp
#6934 NIR Stored: NIR-VIS-NIR-s1_NIR_00137_013.bmp
#6935 NIR Stored: NIR-VIS-NIR-s2_NIR_10046_016.bmp
#6936 NIR Stored: NIR-VIS-NIR-s1_NIR_00044_004.bmp
#6937 NIR Stored: NIR-VIS-NIR-s2_NIR_10202_13-B.bmp
#6938 NIR Stored: NIR-VIS-NIR-s3_NIR_20411_017.bmp
#6939 NIR Stored: NIR-VIS-NIR-s3_NIR_20322_012.bmp
#6940 NIR Stored: NIR-VIS-NIR-s1_NIR_00079_008.bmp
#6941 NIR Stored: NIR-VIS-NIR-s3_NIR_20366_001.bmp
#6942 NIR Stored: NIR-VIS-NIR-s3_NIR_20324_017.bmp
#6943 NIR Stored: NIR-VIS-NIR-s1_NIR_00190_029.bmp
#6944 NIR Stored: NIR-VIS-NIR-s2_NIR_10097_014.bmp
#6945 NIR Stored: NIR-VIS-NIR-s1_NIR_00091_011.bmp
#6946 NIR Stored: NIR-VIS-NIR-s2_NIR_10071_001.bmp
#6947 NIR Stored: NIR-VIS-NIR-s3_NIR_20472_006.bmp
#6948 NIR Stored: NIR-VIS-NIR-s2_NIR_10299_018.bmp
#6949 NIR Stored: NIR-VIS-NIR-s3_NIR_20468_020.bmp
#6950 NIR Stored: NIR-VIS-NIR-s2_NIR_10278_014.bmp
#6951 NIR Stored: NIR-VIS-NIR-

#7093 NIR Stored: NIR-VIS-NIR-s3_NIR_20428_016.bmp
#7094 NIR Stored: NIR-VIS-NIR-s2_NIR_10052_009.bmp
#7095 NIR Stored: NIR-VIS-NIR-s1_NIR_00073_001.bmp
#7096 NIR Stored: NIR-VIS-NIR-s2_NIR_10208_014.bmp
#7097 NIR Stored: NIR-VIS-NIR-s4_NIR_30756_019.bmp
#7098 NIR Stored: NIR-VIS-NIR-s4_NIR_30596_014.bmp
#7099 NIR Stored: NIR-VIS-NIR-s2_NIR_10052_017.bmp
#7100 NIR Stored: NIR-VIS-NIR-s2_NIR_10102_016.bmp
#7101 NIR Stored: NIR-VIS-NIR-s2_NIR_10224_009.bmp
#7102 NIR Stored: NIR-VIS-NIR-s3_NIR_20394_004.bmp
#7103 NIR Stored: NIR-VIS-NIR-s4_NIR_30779_004.bmp
#7104 NIR Stored: NIR-VIS-NIR-s1_NIR_00129_003.bmp
#7105 NIR Stored: NIR-VIS-NIR-s3_NIR_20486_006.bmp
NOT FOUND OR MULTIPLE FACES!
#7106 NIR Stored: NIR-VIS-NIR-s2_NIR_10286_015.bmp
#7107 NIR Stored: NIR-VIS-NIR-s2_NIR_10250_016.bmp
#7108 NIR Stored: NIR-VIS-NIR-s3_NIR_20401_002.bmp
#7109 NIR Stored: NIR-VIS-NIR-s2_NIR_10197_06-B.bmp
#7110 NIR Stored: NIR-VIS-NIR-s1_NIR_00133_004.bmp
#7111 NIR Stored: NIR-VIS-NIR-s2_NIR_10202_16-B.bmp


#7253 NIR Stored: NIR-VIS-NIR-s2_NIR_10192_20-B.bmp
#7254 NIR Stored: NIR-VIS-NIR-s2_NIR_10022_008.bmp
#7255 NIR Stored: NIR-VIS-NIR-s2_NIR_00123_010.bmp
#7256 NIR Stored: NIR-VIS-NIR-s1_NIR_00020_005.bmp
#7257 NIR Stored: NIR-VIS-NIR-s2_NIR_10304_003.bmp
#7258 NIR Stored: NIR-VIS-NIR-s2_NIR_10048_002.bmp
#7259 NIR Stored: NIR-VIS-NIR-s1_NIR_00108_002.bmp
#7260 NIR Stored: NIR-VIS-NIR-s3_NIR_20323_009.bmp
#7261 NIR Stored: NIR-VIS-NIR-s3_NIR_20408_011.bmp
#7262 NIR Stored: NIR-VIS-NIR-s3_NIR_20480_003.bmp
#7263 NIR Stored: NIR-VIS-NIR-s2_NIR_10142_008.bmp
#7264 NIR Stored: NIR-VIS-NIR-s2_NIR_10184_003.bmp
#7265 NIR Stored: NIR-VIS-NIR-s4_NIR_30746_013.bmp
#7266 NIR Stored: NIR-VIS-NIR-s4_NIR_30515_006.bmp
#7267 NIR Stored: NIR-VIS-NIR-s3_NIR_20392_014.bmp
#7268 NIR Stored: NIR-VIS-NIR-s2_NIR_10282_002.bmp
#7269 NIR Stored: NIR-VIS-NIR-s1_NIR_00196_004.bmp
#7270 NIR Stored: NIR-VIS-NIR-s2_NIR_10252_013.bmp
#7271 NIR Stored: NIR-VIS-NIR-s1_NIR_00187_012.bmp
#7272 NIR Stored: NIR-VIS-NIR-

In [39]:
import os
__BUAA_PREPROC_FOLDER = 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR'

buaa_nir_images = []
for root, dirs, files in os.walk(__BUAA_PREPROC_FOLDER):
    for file in files:
        if int(file.split('.')[0]) % 2 == 0:
            buaa_nir_images.append(os.path.join(root, file))

In [41]:
buaa_nir_images

['data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/14.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/4.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/24.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/26.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/18.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/22.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/6.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/12.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/16.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/10.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/28.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/8.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAVISNIR/19/2.bmp',
 'data/Z_PreprocImg-BUAA-centerface-gray-averaged/BUAAV

In [42]:
__AFFECTNET_PREPROC_TRAIN_FOLDER = 'data/AffectNet-8Labels/train_set/images'

retrieve_frist_n_images = len(buaa_nir_images) + len(merged_data['nir_rest'])
affectnet_vis_images = []
for i, f_nms in enumerate(sorted(os.listdir(__AFFECTNET_PREPROC_TRAIN_FOLDER), key=lambda x: int(x.split('.')[0]))):
    if i > retrieve_frist_n_images:
        break
    affectnet_vis_images.append(os.path.join(__AFFECTNET_PREPROC_TRAIN_FOLDER, f_nms))

In [43]:
affectnet_vis_images

['data/AffectNet-8Labels/train_set/images/0.jpg',
 'data/AffectNet-8Labels/train_set/images/1.jpg',
 'data/AffectNet-8Labels/train_set/images/2.jpg',
 'data/AffectNet-8Labels/train_set/images/3.jpg',
 'data/AffectNet-8Labels/train_set/images/5.jpg',
 'data/AffectNet-8Labels/train_set/images/7.jpg',
 'data/AffectNet-8Labels/train_set/images/10.jpg',
 'data/AffectNet-8Labels/train_set/images/13.jpg',
 'data/AffectNet-8Labels/train_set/images/15.jpg',
 'data/AffectNet-8Labels/train_set/images/16.jpg',
 'data/AffectNet-8Labels/train_set/images/18.jpg',
 'data/AffectNet-8Labels/train_set/images/21.jpg',
 'data/AffectNet-8Labels/train_set/images/22.jpg',
 'data/AffectNet-8Labels/train_set/images/23.jpg',
 'data/AffectNet-8Labels/train_set/images/27.jpg',
 'data/AffectNet-8Labels/train_set/images/28.jpg',
 'data/AffectNet-8Labels/train_set/images/29.jpg',
 'data/AffectNet-8Labels/train_set/images/32.jpg',
 'data/AffectNet-8Labels/train_set/images/33.jpg',
 'data/AffectNet-8Labels/train_set/im

In [72]:
import math
a = [0.,0.5]
print(a)

x = math.sqrt(a[0]*a[0] + a[1]*a[1])
print("radius", x, "-", "in" if x <=1 else "out")
r = 1 - x
print("rest to boundary", r)
enl = 1/(1 - a[0])
print("enlarge", enl)
print([a[0]*enl, a[1]*enl])

[0.0, 0.5]
radius 0.5 - in
rest to boundary 0.5
enlarge 2.0
[0.0, 1.0]


In [44]:
os.mkdir('data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir', )
os.mkdir('data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/vis', )
os.mkdir('data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/nir', )

FileExistsError: [Errno 17] File exists: 'data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir'

In [55]:
import shutil

for i, (vis_pth, nir_pth) in enumerate(zip(affectnet_vis_images, buaa_nir_images + preproc_casia_fps)):
#     vis_new_fp = 'data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/vis/' + "affnet-" + vis_pth.split('/')[-1]
    nir_new_fp = 'data/unpaired_additional_train-affectnet_vis_and_buaa_casia_nir/nir/' + "buaa-casia-" + str(i) + "." + (nir_pth.split('/')[-1]).split('.')[-1]
    
#     shutil.copyfile(vis_pth, vis_new_fp)
    shutil.copyfile(nir_pth, nir_new_fp)

In [ ]:
merged_data['nir_rest']